# Annotation errors hava found:

## Minor
- doc_50: full anned as FM
- doc_157: daughte without r

## Big issue
- doc_169 !!
- doc_59: 3 lost last
- doc_66: 4 lost last
- doc_4: 10+ 3 offset 

In [1]:
from os import listdir

directory  = './Data/bioc-FH-training/'
file_list  = listdir(directory)
file_names = set([file.split('.')[0] for file in file_list if file != '' and file.endswith('txt')])

# Read files

In [2]:
import codecs
#import chardet # chardet.detect(binary_var)['encoding']

texts = dict()

for file_name in file_names:
    file = directory + file_name + '.txt'
    
    with open(file, 'r') as f:
        texts[file_name] = f.read().replace('\n', ' ')

# Tokenization and POS

In [3]:
import nltk

sents  = dict()
tokens = dict()
pos    = dict()

for file_name in file_names:
    tokens_   = []
    tags      = []
    
    sentences = nltk.sent_tokenize(texts[file_name])
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        
        tokens_.append(words)
        tags.append(nltk.pos_tag(words))

    sents[file_name] = sentences
    tokens[file_name] = tokens_
    pos[file_name] = tags

In [4]:
# print(tokens)

In [5]:
# print(pos)

# Get annotated entities

In [6]:
import xml.etree.ElementTree

entities = {}
for file_name in file_names:
    file_path = directory + file_name + '.xml'
    
    root = xml.etree.ElementTree.parse(file_path).getroot()
    annotations = root.findall('annotations')[0]

    entities_ = [] # get span of every annoted entity
    for entity in annotations.findall('entity'):
        typee = entity.find('type')

        if typee.text in ['FamilyMember', 'Observation']: # other options: Age, LivingStatus, PHI
            for spans in entity.findall('span')[0].text.split(';'):
                span = list(map(int, spans.split(',')))
                entities_.append((span[0], span[1], typee.text, texts[file_name][span[0]:span[1]]))

    entities_.sort(key=lambda tup: tup[0])
    entities[file_name] = entities_

In [ ]:
num_entities = 0
for file_name in file_names:
    print('\n', file_name)
#     print(texts[file_name])
    for entity in entities[file_name]:
        num_entities += 1
        print(entity)

In [8]:
print("there are", num_entities, "enties in all.")

there are 1787 enties in all.


# Store to pickles

In [9]:
import pickle
from os.path import join

OBJECTS = './objects/'
with open(join(OBJECTS, "texts.pkl"), 'wb') as pkl_file:
    pickle.dump(texts, pkl_file)
with open(join(OBJECTS, "sents.pkl"), 'wb') as pkl_file:
    pickle.dump(sents, pkl_file)
with open(join(OBJECTS, "tokens.pkl"), 'wb') as pkl_file:
    pickle.dump(tokens, pkl_file)
with open(join(OBJECTS, "pos.pkl"), 'wb') as pkl_file:
    pickle.dump(pos, pkl_file)
with open(join(OBJECTS, "entities.pkl"), 'wb') as pkl_file:
    pickle.dump(entities, pkl_file)